# ***TASK #1***

In [ ]:
import psycopg2
conn = psycopg2.connect(dbname='database', user='db_user', 
                        password='mypassword', host='localhost')
cursor = conn.cursor()
text_sql = 'with t1 as ('+
'select * '+
'from сontent_watch '+
'inner join contents '+
'on сontent_watch.content_id = contents.content_id)'+
', t2 as (select distinct '+
	'extract(month from show_date) as months ,'+
	'extract(year from show_date) as years,'+
	'title,'+
	'user_id '+
'from t1)'+
',t3 as('+
'select '+
	'months,years,title,'+
	'count(*) as cnt_shows' +
	'from t2 '+
'group by months,years,title)'+
',t4 as ('+
'select '+
	'months,'+
	'years,'+
	'title,'+
	'cnt_shows,'+
'	row_number() over(partition by months,years order by cnt_shows desc) as rn'+
'from t3)'+
', t5 as ('+
'select * '+
'from t4 '+
'where rn <= 10)'+
',t6 as ('+
'select '+
'	*,'+
	'case when months =1 then 12 else months -1 end as month_before,'+
	'case when months = 1 then (years -1) else years end as years_before'+
'from t5)'+
',t7 as ('+
	'select '+
	'g1.title, g1.cnt_shows, g1.rn, g1.months, g1.years,'+
	'case when g2.cnt_shows is null then 0 else g2.cnt_shows end as cnt_show_before '+
  'from t6 g1 '+
'left join t6 g2 '+
'on g1.month_before = g2.months and g1.years_before = g2.years '+
	'and g1.title = g2.title) '+
'select make_date(cast(years as int), cast(months as int), 1),rn,title,cnt_shows, (cnt_shows-cnt_show_before)/cnt_shows*100 as persentage '+
'from t7'+
'order by rn'
cursor.execute(text_sql)
records = cursor.fetchall()
cursor.close()
conn.close()

# ***TASK #2***

Во втором задании самое адекватное, что смог придумать, это насколько полно посмотрел пользователь тот или иной контент. Выражал этот параметр в процентах, соотвественно в итоговой таблице получил столбец **avg_per** , который показывает, сколько в среднем человек просматривает данный контент в процентном соотношении.


> Второй параметр, который по-моему мнению несет довольно-таки качественную информацию - это сколько людей начинали смотреть данный контент. Так как на  ivi.ru есть как платный доступ к фильмам, так и бесплатный(но в нём присутсвтует реклама, именно от неё зависит прибыль), то было бы логично предположить, наиболее выгодный тот вариант, где рекламу просмотрели наибольшее кол-во раз, а это напрямую зависит от данного параметра(который я получаю вторым **cnt_watch**.Ну и потом я решил их объединить в один параметр **itog** так как предположить какой параметр более значимый, а какой --  менее мы не можем => не можем дать им веса, я решил просто их умножить. 
---



> Из сущностей я бы добавил, это сущность, которая показывала бы, есть в контенте, при данном просмотре реклама или нет, а если есть, то указать через какой промежуток времени появляется(имею ввиду средний показатель), исходя из этих данных, можно было бы более точно указать веса для создания переменной ***itog*** 







In [ ]:
import psycopg2
conn = psycopg2.connect(dbname='database', user='db_user', 
                        password='mypassword', host='localhost')
cursor = conn.cursor()

text_sql = 'with t1 as('+
'select' + 
	'cw.*,c.*, ' +
'	extract(month from show_date) as months, '+
'	extract(year from show_date) as years, '+
'	coalesce(c.compilation_title, c.title) as title_2 '+
'from content_watch cw' '+
'inner join content c '+
'on c.id = cw.content_id) '+
', t2 as(select '+
'	title_2, '+
'	title, '+
'	months, '+
'	years, '+
'	user_id, '+
'	show_duration * 100 / duration as persentage '+
'from t1) '+
', t3 as (select 	 '+
'	title_2, '+
'	title, months,years, avg(persentage) as per_2 '+
'from t2 '+
'group by title_2, title, months, years) '+
', t4 as ( v
'	select title_2, months,years, avg(per_2)  '+
'	from t3 '+
'	group by title_2,  months, years) '+
', t5 as ( '+
'	select  '+
'	title_2, '+
'	months, '+
'	years, '+
'	count(*) as cnt  '+
'from t1 '+
'group by title_2, '+
'	months, '+
'	years) '+
'select t4.title_2 as name_of_serial, '+
'	t4.months,t4.years,round(t4.avg,2) as avg_per, '+
'	t5.cnt as cnt_watch, '+
'	t5.cnt * round(t4.avg,2) as itog '+
'from t4 '+
'inner join t5 '+
'on t4.title_2 = t5.title_2 and t4.months = t5.months and t4.years = t5.years'

cursor.execute(text_sql)
records = cursor.fetchall()
cursor.close()
conn.close()

**Честно, я не до конца разобрался, как совместить PostgreSQL и Jupiter Notebook, вычитал только такой способ, но есть все написанное в PostgreSQL**

# ***TASK #3***

In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
driver =webdriver.Chrome('chromedriver',chrome_options=chrome_options)

In [160]:
wd.get('https://ekinobilet.fond-kino.ru/')
driver.implicitly_wait(10)

In [161]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


Тут я столкнулся с проблемой, так как статистика нужна за последние 3 месяца, то нужно выбирать из выпадающего списка select, но просто через Select у Selenium сделать это не получается, так как option объекты скрыты. Пытался разобраться, так, к сожалению и не получилось :(

In [ ]:
from selenium.webdriver.support.ui import Select

In [162]:
wd.find_element_by_xpath("/html/body/div[@id='hid']/div[@id='main-wrap']/div[@id='main-wrap-inner']/div[@class='wrap']/main[@class='main']/div[@class='two-col-right-side index-page']/div[@class='ts-right']/div[@class='widget-box'][1]/div[@class='wb-body js-tabs']/div[@class='stat-list']/div[@class='sl-date']/div[@id='js-statistic-range-index']/select[@class='js-select sp-select-type']")

<select class="js-select sp-select-type" style="display: none;">

                    <option value="day">День</option>
                            <option value="weekend">Уикенд</option>
                            <option value="week">Неделя</option>
                            <option value="month">Месяц</option>
                            <option value="year">Год</option>
        
    </select>

при парсинге с помощью Selenium получить их можно, но выбрать не получается, вот пример:

In [ ]:
select = Select(wd.find_element_by_xpath("/html/body/div[@id='hid']/div[@id='main-wrap']/div[@id='main-wrap-inner']/div[@class='wrap']/main[@class='main']/div[@class='two-col-right-side index-page']/div[@class='ts-right']/div[@class='widget-box'][1]/div[@class='wb-body js-tabs']/div[@class='stat-list']/div[@class='sl-date']/div[@id='js-statistic-range-index']/select[@class='js-select sp-select-type']"))
select.select_by_visible_text('Месяц')

В принципе в этом и была загвосдка, так как остальные данные парсятся легко, и сам EDA, а так же визуализация уже не столь сложная задача


---
Пробовал и кликать на блок, с которым связан select, но так и не получилось, вот пример:

In [135]:
wd.find_element_by_xpath("//div[@class='sl-date']/div[@id='js-statistic-range-index']/span[@class='js-custom-select-4-box']/span[@class='js-custom-select']/span[@class='js-custom-select-value']").click()
time.sleep(5)
select = Select(wd.find_element_by_xpath("//div[@class='sl-date']/div[@id='js-statistic-range-index']/select[@class='js-select sp-select-type']"))
select.all_selected_options

[<option value="day">День</option>]

А так, вот пример парсинга данных, но тут парсинг идет по дням, что не подходит для поставленной задачи, как и говорил выше проблема заключалась именно в смене показа данных со стандартного(день) на нужный нам(месяц)

In [40]:
from bs4 import BeautifulSoup
html = BeautifulSoup(wd.page_source)

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(); 

In [8]:
li_list = html.find('ul', attrs={'class':'sl-list'}).find_all('li')

In [14]:
name = li_list[0].find('span',attrs = {'class':'sl-item__name'}).text
fees = int(li_list[0].find('span',attrs = {'class':'-fees'}).text.replace(" ", "").replace("\n",""))
spectators = int(li_list[0].find('span',attrs = {'class':'-spectators'}).text.replace(" ", "").replace("\n",""))
sessions = int(li_list[0].find('span',attrs = {'class':'-sessions'}).text.replace(" ", "").replace("\n",""))
hours_currency = int(li_list[0].find('span',attrs = {'class':'-hours-currency'}).text.replace(" ", "").replace("\n",""))
hours_user = int(li_list[0].find('span',attrs = {'class':'-hours-user'}).text.replace(" ", "").replace("\n",""))

Вот основываясь на этих данных уже был бы создан DataFrame, который дальше бы помог в задаче EDA

***Ещё обнаружилось, что фонд кино, а именно эта страница, обновляет свои данные примерно с 00:00, до 3:00, пропадают все цифры***